In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [2]:
from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime, set_option
from TpsParse.Tps.TpsFile import TpsFile
import os
from numpy import quantile, nan
from math import ceil
from math import pi

set_option('display.max_columns', 500)
set_option('display.max_row', 1000)

from bokeh.resources import CDN
from bokeh.embed import components
from bokeh.models import Range1d, NumeralTickFormatter, ColumnDataSource, FactorRange, HoverTool
from bokeh.plotting import figure
from bokeh.transform import linear_cmap, factor_cmap, cumsum
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20_20

from django.db.models import Sum, DateField
from django.db.models.functions import Trunc
from Utils.clean_string import clean_string

output_notebook()

Loading BokehJS ...

In [3]:
def parser(path):
    file = open(path, "r+b")
    tps = TpsFile(file)

    cols_and_type = []
    columns = []
    records = []
    for definition in tps.get_table_definitions():
        for field in definition.fields:
            column = field.field_name.split(':')[1].lower()
            col_and_type = (field.field_name.split(':')[1].lower(), field.type)
            if column not in columns:
                columns.append(column)
                cols_and_type.append(col_and_type)
        for record in tps.get_data_records(definition):
            records.append([r.strip().title() if isinstance(r, str) else r for r in record.values])

    table = []
    for record in records:
        dic = {}
        for col, rec in zip(columns, record):
            dic[col] = rec
        table.append(dic)

    df = DataFrame(table, columns=columns)

    name_list = ('fec', 'periodo')
    # Formatear la fecha.
    date_columns = [cname for cname, ctype in cols_and_type if cname.startswith(name_list) and ctype == 'SignedLong']
    for col in df[date_columns]:
        df[col] = datetime(1800, 12, 28) + df[col].map(timedelta)
        df.loc[df[col] == '1800-12-28', col] = None

    # Reemplazar los campos vacíos con None.
    df.replace(to_replace='', value=nan, inplace=True)
    df.replace(to_replace=0, value=None, inplace=True)
    
    # Borrar los registros que tengan todos valores nulos.
    df.dropna(axis='index', how='all', inplace=True)
    
    # Eliminar registros duplicados.
    df.drop_duplicates(keep='last', inplace=True)
    
    # Borrar las columnas que tengan todos valores nulos o ceros.
    df.dropna(axis='columns', how='all', inplace=True)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [63]:
from app.models import Movimpu

movimpu = Movimpu.objects.values(
    'cod_imput',
    'cod_imput__descripcion',
    'cod_imput__donde_acumula__descripcion',
    'fecha_comprob',
    'periodo_contable',
    'tipo_movim',
    'num_comprob',
    'descripcion',
    'importe',
    'actividad__descripcion')\
.filter(importe__gt=0)\
.order_by('periodo_contable')

df = DataFrame(movimpu)
df = df.rename({
    'actividad__descripcion': 'Actividad',
    'fecha_comprob': 'Fecha Comprobante',
    'nro_comprob': 'Numero Comprobante',
    'periodo_contable': 'Periodo Contable',
    'tipo_movim': 'Tipo Movimiento',
    'descripcion': 'Descripción',
    'importe': 'Importe',
    'cod_imput': 'Codigo Imputación',
    'cod_imput__descripcion': 'Imputación',
    'cod_imput__donde_acumula__descripcion': 'Dónde Acumula',
},
    axis='columns')
df['Periodo Contable'] = to_datetime(df['Periodo Contable'])
df['Fecha Comprobante'] = to_datetime(df['Fecha Comprobante'])
# Diferenciar ingresos de egresos
df['Tipo'] = 'Egresos'
df.loc[(df['Tipo Movimiento'] == 'Cobranza') | (
        df['Tipo Movimiento'] == 'Nota De Crédito'), 'Tipo'] = 'Ingresos'

In [57]:
df

,Codigo Imputación,Imputación,Dónde Acumula,Fecha Comprobante,Periodo Contable,Tipo Movimiento,num_comprob,Descripción,Importe,Actividad,Tipo
0,2.0,Pago Proveedores,Administracion,2002-05-20,2002-05-20,Orden De Pago,3444,Fundacion Castellanos,1349.6,None,Egresos
1,30.0,Tambero,Servicios Agrop Tambo,2005-05-11,2005-05-31,Facturas Cuenta Corriente,29,Maria Cristina Minotti,18913.0,None,Egresos
2,2.0,Pago Proveedores,Administracion,2011-10-20,2011-10-20,Orden De Pago,1565,Sucesion Heck Eldo Olivio,6231.5,None,Egresos
3,2.0,Pago Proveedores,Administracion,2011-10-25,2011-10-25,Orden De Pago,1457,Erio Gonzalo Rodriguez,350.0,None,Egresos
4,2.0,Pago Proveedores,Administracion,2011-11-20,2011-11-20,Orden De Pago,1553,Dominga B. Alemandi,11760.0,None,Egresos
...,...,...,...,...,...,...,...,...,...,...,...
9588,6.0,Alquiler Campo Agricultura,Alquiler Rural,2017-06-19,2017-06-30,Facturas Cuenta Corriente,97,Teresita Franquina Racca,76000.0,None,Egresos
9589,75.0,Alquiler Vaca Para Tambo,Insumos Tambo,2017-06-21,2017-06-30,Facturas Cuenta Corriente,105,Analia G. Bonafede,11059.5,None,Egresos
9590,50.0,Ventas Animales,Ventas Animales,2017-06-08,2017-06-30,Ventas Cuenta Corriente,32,Racca Jose Luis,14765.0,Ganaderia,Egresos
9591,105.0,Deudores Por Ventas,Administracion,2017-11-30,2017-11-30,Cobranza,483,Rene Y Mario Mina,6310.0,None,Ingresos


In [70]:
from datetime import datetime

In [78]:
today = datetime(2014,4,1)

In [81]:
df_filter = df[(df['Periodo Contable'].dt.month == today.month) & (df['Periodo Contable'].dt.year == today.year)]

In [104]:
df_gb = DataFrame(df_filter.groupby(['Periodo Contable', 'Tipo', 'Tipo Movimiento', 'Dónde Acumula', 'Imputación'], as_index=False).Importe.sum())

In [105]:
df_gb

,Periodo Contable,Tipo,Tipo Movimiento,Dónde Acumula,Imputación,Importe
0,2014-04-01,Egresos,Documento De Contado,Gastos Varios,Gastos Bancarios,125.00
1,2014-04-01,Egresos,Documento De Contado,Impuestos,Comision Mant. De Cuenta,208.70
2,2014-04-01,Egresos,Documento De Contado,Impuestos,Imp Ley 25413 Afip,4.44
3,2014-04-01,Egresos,Facturas Cuenta Corriente,Alquiler Rural,Alquiler Campo Tambo,68654.40
4,2014-04-01,Egresos,Facturas Cuenta Corriente,Combustible,Aceites,3077.00
5,2014-04-01,Egresos,Facturas Cuenta Corriente,Ganaderia,Compra De Animales,16800.00
6,2014-04-01,Egresos,Facturas Cuenta Corriente,Insumos Agricultura,Agroquimicos,34996.25
7,2014-04-01,Egresos,Facturas Cuenta Corriente,Insumos Agricultura,Semilla De Avena,10507.80
8,2014-04-01,Egresos,Facturas Cuenta Corriente,Insumos Tambo,Alquiler Vaca Para Tambo,18504.00
9,2014-04-01,Egresos,Facturas Cuenta Corriente,Insumos Tambo,Balanceado,65750.00


In [108]:
source=ColumnDataSource(df_gb)
source_ingresos = ColumnDataSource(df_gb[df_gb['Tipo'] == 'Ingresos'])
source_egresos = ColumnDataSource(df_gb[df_gb['Tipo'] == 'Egresos'])
tooltips = [("Tipo", "@Tipo"),
    ("Dónde Acumula", "@{Dónde Acumula}"),
            ("Imputación", "@{Imputación}"),
            ("Tipo Movimiento", "@{Tipo Movimiento}"),
            ("Importe total", "@Importe{$0.00}")]
hovertool = HoverTool(names=['circle'], tooltips=tooltips)
plot = figure(
    tools=["tap,reset,save,crosshair", hovertool],
    toolbar_location='above',
    x_axis_type="datetime",
    y_range = Range1d(start=0,
                      end=max(source.data['Importe']), 
                      bounds='auto')
)
plot.line(x='Periodo Contable', y='Importe',
           source=source_ingresos, name='line',
         line_color='green')
plot.circle(x='Periodo Contable', y='Importe', 
            fill_color="white", size=8, name='circle', 
            source=source_ingresos,
           line_color='green')
plot.line(x='Periodo Contable', y='Importe',
           source=source_egresos, name='line',
         line_color='red')
plot.circle(x='Periodo Contable', y='Importe', 
            fill_color="white", size=8, name='circle', 
            source=source_egresos,
           line_color='red')
plot.sizing_mode = 'scale_both'
show(plot)

In [101]:
max(source_ingresos.data['Importe'], source_egresos.data['Importe'])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()